In [19]:
import numpy as np
import pandas as pd
np.random.seed(8)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('train.csv')
test_cm = pd.read_csv('test.csv')
test_lb = pd.read_csv('test_labels.csv')
test_all = pd.merge(test_cm, test_lb, on='id')
test = test_all[test_all['toxic'] != -1]

In [20]:
X_train = train["comment_text"].str.lower()
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].str.lower()
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [21]:
vocab = 100000
maxlen = 200
embed_size = 300

In [22]:
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train) + list(X_test))

In [23]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [24]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))


Loaded 2000000 pre-trained words


In [25]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [26]:
def make_model(k=[2,3,5,7], activation='tanh', filters=32, Sdroprate=0.4, droprate=0.0):

    inp = Input(shape=(maxlen, ))
    x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
    x = SpatialDropout1D(Sdroprate)(x)
    x = Reshape((maxlen, embed_size, 1))(x)
    
    conv = dict()
    maxpool = dict()
    for h in k:
        conv[h] = Conv2D(filters, kernel_size=(h, embed_size), activation=activation)(x)
        maxpool[h] = MaxPool2D(pool_size=(maxlen - h + 1, 1))(conv[h])
        
    y = Concatenate(axis=1)([pool for key,pool in maxpool.items()])
    y = Flatten()(y)
    y = Dropout(droprate)(y)

    outp = Dense(6, activation="sigmoid")(y)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
model = make_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 300)     30000000    input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 200, 300)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 200, 300, 1)  0           spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
conv2d_5 (

In [28]:
class RocAuc(Callback):

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0], verbose=0)
        score = roc_auc_score(self.validation_data[1], y_pred)
        print("\n roc_auc - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [29]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
batch_size=256
epochs = 5
file_path = 'saved_models/weights.best.hdf5'
check_point = ModelCheckpoint(filepath=file_path, verbose=1, save_best_only=True)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
roc_auc = RocAuc()
x_trainS, x_val, y_trainS, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=8)

In [30]:
h = model.fit(x_trainS, y_trainS, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val),
                 callbacks=[check_point, early_stop, roc_auc], verbose=2)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
 - 702s - loss: 0.0815 - acc: 0.9744 - val_loss: 0.0469 - val_acc: 0.9824

Epoch 00001: val_loss improved from inf to 0.04690, saving model to saved_models/weights.best.hdf5

 roc_auc - epoch: 1 - score: 0.980486 

Epoch 2/5
 - 695s - loss: 0.0487 - acc: 0.9818 - val_loss: 0.0429 - val_acc: 0.9835

Epoch 00002: val_loss improved from 0.04690 to 0.04285, saving model to saved_models/weights.best.hdf5

 roc_auc - epoch: 2 - score: 0.985957 

Epoch 3/5
 - 688s - loss: 0.0449 - acc: 0.9828 - val_loss: 0.0416 - val_acc: 0.9842

Epoch 00003: val_loss improved from 0.04285 to 0.04163, saving model to saved_models/weights.best.hdf5

 roc_auc - epoch: 3 - score: 0.987290 

Epoch 4/5
 - 666s - loss: 0.0428 - acc: 0.9836 - val_loss: 0.0411 - val_acc: 0.9844

Epoch 00004: val_loss improved from 0.04163 to 0.04113, saving model to saved_models/weights.best.hdf5

 roc_auc - epoch: 4 - score: 0.988055 

Epoch 5/5
 - 679s - loss: 0.0412 - ac

In [31]:
y_pred = model.predict(x_test, batch_size=256)

In [32]:
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))


 roc_auc score: 0.982194 



In [33]:
from keras.models import load_model
model_saved = load_model(file_path)

In [34]:
model_saved.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 300)     30000000    input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 200, 300)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 200, 300, 1)  0           spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
conv2d_5 (

In [35]:
y_pred = model_saved.predict(x_test, batch_size=256)

In [36]:
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))


 roc_auc score: 0.981582 

